# Imports

In [22]:
from geopy.geocoders import GoogleV3
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd
import numpy as np
import urllib3
import re
from textwrap import shorten
import os

In [23]:
df = pd.read_csv('data_with_geo_points.csv')

In [24]:
len(df)

659

In [25]:
def popup_html(row):
    i = row
    address = df['full_address'].iloc[i]
    architect = df['name'].iloc[i]
    work_desc = df['work_desc'].iloc[i]
    issue_date = df['issue_date'].iloc[i]
    
    html = '''<!DOCTYPE html>
    <html>
    Address: <strong>{}'''.format(address) + '''</strong><br>
    Architect: <strong>{}'''.format(architect) + '''</strong><br>
    Description: <strong>{}'''.format(work_desc) + '''</strong><br>
    Issue Date: <strong>{}'''.format(issue_date) + '''</strong><br>
    </html>
    '''
    return html

## No popup or Tooltip - Pins Only

In [26]:
import folium
import branca

f = folium.Figure(width=750, height=750)
m = folium.Map(location=df[["lat", "lon"]].mean().to_list(),zoom_start=10)

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Chicago\'s most influential architects')

for i in range(0,len(df)):
    popup = folium.Popup(folium.Html(html, script=True))
    folium.Marker([df['lat'].iloc[i],df['lon'].iloc[i]]).add_to(m)

m.get_root().html.add_child(folium.Element(title_html))
m.fit_bounds(bounds=df[['lat','lon']].mean().to_list(),max_zoom=15)

m

## Popup attempt

In [27]:
import folium
import branca

f = folium.Figure(width=750, height=750)
m = folium.Map(location=df[["lat", "lon"]].mean().to_list(),zoom_start=8)

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Chicago\'s most influential architects')

for i in range(0,len(df)):
    html = popup_html(i)
    iframe = branca.element.IFrame(html=html)
    popup = folium.Popup(folium.Html(html, script=True))
    folium.Marker([df['lat'].iloc[i],df['lon'].iloc[i]],
                 popup=popup).add_to(m)

m.get_root().html.add_child(folium.Element(title_html))
m.fit_bounds(bounds=df[['lat','lon']].mean().to_list(),max_zoom=15)

m

## Tooltip attempt

In [28]:
import folium
from folium.plugins import MarkerCluster
import branca

f = folium.Figure(width=750, height=750)
m = folium.Map(location=df[["lat", "lon"]].mean().to_list(),zoom_start=8)

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Chicago\'s most influential architects')

marker_cluster = MarkerCluster().add_to(m)

for i, r in df.iterrows():
    location = (r['lat'], r['lon'])
    info = (r['full_address'],r['name'],r['work_desc'],r['issue_date'])
    info = list(info)
    new_line = '<br>'
    bold_start = '<strong>'
    bold_end = '</strong>'
    text = f'Address: {bold_start}{info[0]}{bold_end}{new_line}\
    Architect: {bold_start}{info[1]}{bold_end}{new_line}\
    Description: {bold_start}{info[2]}{bold_end}{new_line}\
    Issue Date: {bold_start}{info[3]}{bold_end}{new_line}'
    folium.Marker(location=location,tooltip=text).add_to(marker_cluster)

m.get_root().html.add_child(folium.Element(title_html))
m.fit_bounds(bounds=df[['lat','lon']].mean().to_list(),max_zoom=15)

m